# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [1]:
import os
os.environ['OPENAI_API_KEY'] = 'XXX'
os.environ['OPENAI_BASE_URL'] = 'XXX'

In [3]:
import os
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图展示了一条木制的栈道穿过一片郁郁葱葱的草地。栈道直线延伸，视角引人深入画面，引导观者的视线向画面中的远方探去。四周的草地呈现出生机勃勃的绿色，较高的草丛两侧紧邻着栈道，增添了一种自然野趣。天空蓝得清澈，布满了细碎的白云，增添了一种宁静和广阔的感觉。\n\n整体上，这幅图传达了一种平和与自然美的氛围，是户外活动和自然探索的完美象征。这样的场景可能位于一个自然保护区或是公园内，为游客提供了一条安全而舒适的步行路径，可以近距离地感受自然的美丽与宁静。', role='assistant', function_call=None, tool_calls=None))


In [4]:
response.choices[0].message.content

'这幅图展示了一条木制的栈道穿过一片郁郁葱葱的草地。栈道直线延伸，视角引人深入画面，引导观者的视线向画面中的远方探去。四周的草地呈现出生机勃勃的绿色，较高的草丛两侧紧邻着栈道，增添了一种自然野趣。天空蓝得清澈，布满了细碎的白云，增添了一种宁静和广阔的感觉。\n\n整体上，这幅图传达了一种平和与自然美的氛围，是户外活动和自然探索的完美象征。这样的场景可能位于一个自然保护区或是公园内，为游客提供了一条安全而舒适的步行路径，可以近距离地感受自然的美丽与宁静。'

### 封装成一个函数 query_image_description

In [6]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI()  # 初始化 OpenAI 客户端
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [7]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这幅图是一种幽默的表现形式，对比了两种不同的生活方式或成长环境对生物（在这种情况下是狗）的影响。左侧的图片展示了一只具有人类健身者体型的大型肌肉狗，这种描绘方法通常被用来幽默地表达极端的锻炼和饮食控制。图中文字解释说，这只狗每日锻炼、拥有良好的自律，只吃高蛋白食物，并且睡眠充足。

右侧的图片则展示了一只看起来较为普通、稍显发福的狗，坐着的样子看起来很悠闲。文字内容似乎表明这只狗的生活方式相对自由随意，不太注重锻炼，也无需严格控制饮食。

这样的对比通常用来幽默地讨论和对


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [34]:
from openai import OpenAI
import base64
import requests
import json

client = OpenAI()  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://XXX/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [19]:
content = query_base64_image_description("./images/gdp_1980_2020.jpg")
print(content)

这张图显示了从1980年到2020年之间美国、中国、日本和德国的国内生产总值（GDP）的对比。图中使用不同的颜色来代表不同的国家：

- **蓝色线条代表美国**：从1980年开始，美国的GDP逐年增长，到2020年达到超过20万亿美元。
- **红色线条代表中国**：1980年前中国的GDP较低，但从1990年代开始迅速增长，尤其是进入2000年后，增长更为显著，到2020年接近美国的GDP。
- **紫色线条代表日本**：日本的GDP在1980年代增长很快，但进入1990年代后增速放缓，2000年以后基本维持在一个较为平稳的水平。
- **绿色线条代表德国**：德国的GDP增长相较于美国和中国较为平稳，没有出现特别大的波动。

整体而言，这张图展示了过去40年四个主要经济体的经济增长趋势和差异。其中中国的迅猛增长尤为显眼，而美国则保持其经济总量的全球领先地位。


#### 使用 GPT-4V 识别手写体笔记

![](./images/handwriting_0.jpg)

In [20]:
content = query_base64_image_description("./images/handwriting_0.jpg")
print(content)

这张图片显示的是一本笔记本上关于机器学习(尤其是自然语言处理)的一些笔记。主要内容涉及Prompt Tuning（诱导调整）、Prefix Tuning（前缀调整）以及LoRA（Low-Rank Adaptation，低秩适应）的模型调整策略。

1. **Prompt Tuning (FMT & Small Model)**：诱导调整常用于小模型上。这里的X表示输入序列`[X1, X2, ..., Xn]`转成的Embedding，然后再添加可以学习的诱导Token进行训练，通过这种方式来调整模型行为。

2. **Prefix Tuning**：前缀调整是通过在输入序列之前添加一系列可学习的前缀（`W'=[W_p; W_L]`），将这些前缀与输入序列结合，并通过Transformer的Encoder/Decoder来处理，生成输出`.

3. **LoRA (Low-Rank Adaptation)**：低秩适应是通过对模型的权重矩阵W进行修改，使用增量方式 `ΔW = AB`（其中A和B是可以学习的参数矩阵）来改进训练效率和模型性能。原文中提到了如何调整这个权重矩阵`(W+ΔW)`来适应特定的任务需求。

文章还提到了一些关于这些技术的存储占用量，比如LoRA技术的存储占用相对较大（约78GB），而优化后的Q-LORA方法可以降低到48GB。

这些笔记反映了一些先进的技术在处理深度学习和自然语言处理任务中的应用方法及其效率和性能改进策略。


#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [35]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

<IPython.core.display.Markdown object>

![](./images/handwriting_1.jpg)

In [36]:
content = query_base64_image_description("./images/handwriting_1.jpg")
display(Markdown(content))

这张图展示了一个笔记本上的手写笔记，内容涉及人工智能领域中的变换器（Transformers）模型和各种训练技术，尤其是在自然语言处理（NLP）中的应用。笔记包括了多种技术和方法的说明，包括：

1. **P-EFT/SOTA (State of the Art)** 和 **PBFT Methods**：这些可能指的是高级细粒度调整技术（尽管缩写的具体意义不完全清楚），SOTA表示行业内的最高标准或者技术。

2. **Prompt Tuning**：是一种调整预训练语言模型的技术，强调通过提示（prompt）的方式来调优模型，使其更好地适应特定的任务。

3. **Adapter**、**Prefix**、**P-Tuning V1/V2** 和 **Soft Prompts**：这些都是用于调整预训练语言模型的不同技术，通常旨在减少必要调整的参数数量，从而高效地进行模型微调。

4. **Llama (L3B), PETC, LoRA, GLaRA** 等：似乎提到了不同的模型架构或者技术。

5. **MAM Adapters**：可能是指一种特定的适配器技术，用于模型细粒度的调整。

6. **Instruction Tuning** 和 **FT-LLMs**：关注在大语言模型上采用指令式调优。

7. **特定模型名称**，如 **LaMDA**, **GPT-X, LLM, ChatGPT**, **BLOOM**, **Alpaca**, **MOSS**：这些是不同的语言模型或框架，广泛应用于各种NLP任务中。

整体上，这张笔记反映了笔者对最新的NLP模型调整技术和方法的学习和总结。

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。

In [5]:
from openai import OpenAI
import base64
import requests
from IPython.display import Markdown

client = OpenAI()  # 初始化 OpenAI 客户端

def recognize_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://XXX/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return Markdown(content)
    else:
        return f"Error: {response.status_code}, {response.text}"

In [6]:
from IPython.display import display
content = recognize_image_description("./images/gpt-4v.jpg")
display(content)

这张图片展示的是一张手写的笔记或规划文件，内容涉及日程安排、活动或工作流程。笔记分为多个部分，每部分具体说明了时间、活动内容与步骤。这些笔记有可能是为了组织某个事件、会议或项目的时间表。

例如，笔记中涉及的具体安排包括：

1. 时间安排，比如在哪一天的哪个时间段进行哪些活动。
2. 活动内容，涉及会议、讨论等。
3. 活动地点或相关的人员信息可能也包含在内，但在这张图片中没有明确显示。

此笔记的内容看起来是针对具体的操作和实施步骤进行详细规划，可能是为了确保每一个环节都能按时按质完成。整体上显示了良好的组织和计划性。